In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# time
import datetime

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

#### ukb46987_new에서 Heart Failure (I50) 진단자, 진단일 열 생성     
- MI는 UKB_preprocess 안에 있음

In [2]:
# 8분 이상 걸림

ukb46987 = pd.read_sas('SAS\\ukb46987_new.sas7bdat', format = 'sas7bdat', encoding = 'cp1252')
ukb46987

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = ukb46987.copy()

In [4]:
df2 = df.iloc[:, :224]
df2

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41270-0.213,41270-0.214,41270-0.215,41270-0.216,41270-0.217,41270-0.218,41270-0.219,41270-0.220,41270-0.221,41270-0.222
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df4 = df.iloc[:, 224:]
df4

,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1999-07-22,2018-06-07,2002-10-30,1999-09-23,2002-10-30,2002-10-17,2002-10-30,2018-06-07,2018-06-07,2014-12-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-16,2014-10-24,2014-10-24,2015-01-16,2014-10-24,2004-05-27,2004-05-27,2015-01-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-13,2018-10-16,2018-10-30,2018-11-20,2019-02-05,2020-02-13,2018-10-09,2018-10-09,2018-11-20,2018-10-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,1999-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,2014-09-30,2014-09-30,2014-09-30,2014-09-30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,2013-12-30,2013-12-30,2013-12-30,2013-12-30,2015-09-30,2015-09-30,2013-12-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,1999-01-25,2015-10-17,2002-03-24,2002-04-06,2002-04-06,2015-10-17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# colnames = df2.columns.tolist()
# colnames.remove('eid')

# n18 = 0
# for row in range(len(df2)): 
#     for col in colnames: 
#         print(df2[col][row])
#         if df2[col][row] is not np.nan: 
#             disease = df2[col][row].decode('euc-kr')
#             if disease.startswith('N18') == True: 
#                 n18 += 1
#                 break

In [17]:
TT = pd.DataFrame(df2['41270-0.0'].loc[(df2['41270-0.0'].str.startswith('H400')) & (df2['41270-0.0'].isna() == False)])

In [6]:
# df2['H400'] = 0
df2['I500'] = 0
df2['I501'] = 0
df2['I509'] = 0

In [7]:
df2

,eid,41270-0.0,41270-0.1,41270-0.2,41270-0.3,41270-0.4,41270-0.5,41270-0.6,41270-0.7,41270-0.8,...,41270-0.216,41270-0.217,41270-0.218,41270-0.219,41270-0.220,41270-0.221,41270-0.222,I500,I501,I509
0,1000015,D693,E119,E780,I10,I200,I209,I251,I259,I350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
1,1000027,D128,E780,J459,K621,N40,S609,W290,Z800,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,1000039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
3,1000040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,1000053,D127,D508,D509,D649,E669,F412,F431,G933,I10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,I839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
502472,6025340,E039,Z038,Z538,Z873,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
502473,6025353,I083,I10,I278,I48,S9230,W199,Z867,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
502474,6025364,J189,K573,N393,T814,Y838,Z121,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0


In [8]:
colnames = df2.columns.tolist()
colnames.remove('eid')

In [55]:
# for col in colnames: 
#     df2.loc[df2[col] == 'H400', 'H400'] = 1  # 전체 222개 칼럼 순회하면서 H400 있으면 1 넣어주기
#     df2.loc[df2[col] == 'H400', 'H400_index'] = col[6:]  # 41280 변수에서 H40 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

#     #df2["N180"] = df2.apply(lambda x: 1 if x[col] == 'N180' else 0, axis=0)

In [56]:
# df2['H400'].value_counts() 

0    499527
1      2949
Name: H400, dtype: int64

In [57]:
# # 인덱스랑 진단여부 갯수 똑같은 거 확인

# df2['H400_index'].isna().sum()

499527

In [9]:
for col in colnames: 
    df2.loc[df2[col] == 'I500', 'I500'] = 1  # 전체 222개 칼럼 순회하면서 I500 있으면 1 넣어주기
    df2.loc[df2[col] == 'I500', 'I500_index'] = col[6:]  # 41280 변수에서 I50 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [10]:
for col in colnames: 
    df2.loc[df2[col] == 'I501', 'I501'] = 1  # 전체 222개 칼럼 순회하면서 I501 있으면 1 넣어주기
    df2.loc[df2[col] == 'I501', 'I501_index'] = col[6:]  # 41280 변수에서 I50 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [11]:
for col in colnames: 
    df2.loc[df2[col] == 'I509', 'I509'] = 1  # 전체 222개 칼럼 순회하면서 I509 있으면 1 넣어주기
    df2.loc[df2[col] == 'I509', 'I509_index'] = col[6:]  # 41280 변수에서 I50 진단받은 위치와 동일한 위치에 있는 날짜 뽑아오기 위해 칼럼명을 잘라서 인덱스만 가져오기

In [12]:
df2["I500"].value_counts()

0    496790
1      5686
Name: I500, dtype: int64

In [13]:
df2['I500_index'].isna().sum()

496790

In [14]:
df2["I501"].value_counts()

0    494949
1      7527
Name: I501, dtype: int64

In [15]:
df2['I501_index'].isna().sum()

494949

In [16]:
df2["I509"].value_counts()

0    497634
1      4842
Name: I509, dtype: int64

In [17]:
df2['I509_index'].isna().sum()

497634

In [18]:
# df_final = df2[['eid', 'H400', 'H401', 'H402', 'H403', 'H404', 'H405', 'H406', 'H407', 'H408', 'H409',  
#                 'H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index', 
#                 ]]
# df_final

df_final = df2[['eid', 'I500', 'I501', 'I509', 
                'I500_index', 'I501_index', 'I509_index'
                ]]
df_final

,eid,I500,I501,I509,I500_index,I501_index,I509_index
0,1000015,1,0,1,0.11,NaN,0.12
1,1000027,0,0,0,NaN,NaN,NaN
2,1000039,0,0,0,NaN,NaN,NaN
3,1000040,0,0,0,NaN,NaN,NaN
4,1000053,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN
502472,6025340,0,0,0,NaN,NaN,NaN
502473,6025353,0,0,0,NaN,NaN,NaN
502474,6025364,0,0,0,NaN,NaN,NaN


In [56]:
df4

,41280-0.0,41280-0.1,41280-0.2,41280-0.3,41280-0.4,41280-0.5,41280-0.6,41280-0.7,41280-0.8,41280-0.9,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1999-07-22,2018-06-07,2002-10-30,1999-09-23,2002-10-30,2002-10-17,2002-10-30,2018-06-07,2018-06-07,2014-12-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-16,2014-10-24,2014-10-24,2015-01-16,2014-10-24,2004-05-27,2004-05-27,2015-01-16,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-13,2018-10-16,2018-10-30,2018-11-20,2019-02-05,2020-02-13,2018-10-09,2018-10-09,2018-11-20,2018-10-09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,1999-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,2014-09-30,2014-09-30,2014-09-30,2014-09-30,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,2013-12-30,2013-12-30,2013-12-30,2013-12-30,2015-09-30,2015-09-30,2013-12-30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,1999-01-25,2015-10-17,2002-03-24,2002-04-06,2002-04-06,2015-10-17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df5 = pd.concat([df_final, df4], axis = 1)
df5

,eid,I500,I501,I509,I500_index,I501_index,I509_index,41280-0.0,41280-0.1,41280-0.2,...,41280-0.213,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222
0,1000015,1,0,1,0.11,NaN,0.12,1999-07-22,2018-06-07,2002-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,NaN,NaN,NaN,2015-01-16,2014-10-24,2014-10-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,NaN,NaN,NaN,2020-02-13,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN,1999-06-10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,NaN,NaN,NaN,2014-09-30,2014-09-30,2014-09-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,NaN,NaN,NaN,2013-12-30,2013-12-30,2013-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,NaN,NaN,NaN,1999-01-25,2015-10-17,2002-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# dftmp = df5[['H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']].copy()
dftmp = df5[['I500_index', 'I501_index', 'I509_index']].copy()

In [92]:
# dftmp['H400_index'].unique()

array([nan, '0.15', '0.2', '0.6', '0.3', '0.0', '0.13', '0.7', '0.1',
       '0.5', '0.4', '0.11', '0.10', '0.8', '0.9', '0.18', '0.12', '0.14',
       '0.19', '0.21', '0.17', '0.16', '0.20', '0.22', '0.23', '0.24'],
      dtype=object)

In [ ]:
#### TEST

coltest = ['idx1', 'idx2', 'idx3', '41280-1.0', '41280-5.0', '41280-9.0']
value = [[1, np.nan, np.nan, '2010-01-01', np.nan, np.nan], [np.nan, 5, np.nan, np.nan, '2010-03-03', np.nan], [np.nan, np.nan, 9, np.nan, np.nan, '2021-01-01'], [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]]
testdf = pd.DataFrame(columns = coltest, data=value)

# vltest = testdf[col].sum(axis = 1).tolist()
# vltestdf = pd.DataFrame(vltest)
# vltestdf[0].unique()

testdf['N18_idx'] = testdf.stack().groupby(level=0).first().reindex(testdf.index)   # 각 행에서 Nan 값이 아닌 유일한 값 가져오기
testdf
# testdf['N18_Date'] = list(map(lambda y: np.nan if y is np.nan else testdf['41280-' + str(y)].values[0], testdf['N18_idx']))

In [ ]:
testdf['eid'] = ['A', 'B', 'C', 'D']
valuelist = testdf['N18_idx'].tolist()

testdf['N18_Date'] = list(map(lambda x, y: np.nan if y is np.nan else testdf.loc[testdf['eid'] == x, '41280-' + str(y)].values[0], testdf['eid'], valuelist))

In [ ]:
testdf

In [ ]:
########## test end

In [21]:
# column_list = ['H400_index', 'H401_index', 'H402_index', 'H403_index', 'H404_index', 'H405_index', 'H406_index', 'H407_index', 'H408_index', 'H409_index']

# dftmp['H40_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

column_list = ['I500_index', 'I501_index', 'I509_index']

dftmp['I50_idx'] = dftmp.stack().groupby(level=0).first().reindex(dftmp.index)  # 각 행에서 Nan 값이 아닌 유일한 값 가져오기

In [22]:
dftmp['I50_idx'].unique()

array(['0.11', nan, '0.16', '0.23', '0.8', '0.6', '0.9', '0.2', '0.22',
       '0.13', '0.15', '0.18', '0.4', '0.12', '0.5', '0.7', '0.14',
       '0.20', '0.1', '0.3', '0.21', '0.10', '0.17', '0.24', '0.19',
       '0.0'], dtype=object)

In [23]:
dftmp

,I500_index,I501_index,I509_index,I50_idx
0,0.11,NaN,0.12,0.11
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
502471,NaN,NaN,NaN,NaN
502472,NaN,NaN,NaN,NaN
502473,NaN,NaN,NaN,NaN
502474,NaN,NaN,NaN,NaN


In [24]:
df5 = pd.concat([df5, dftmp['I50_idx']], axis = 1)
df5

,eid,I500,I501,I509,I500_index,I501_index,I509_index,41280-0.0,41280-0.1,41280-0.2,...,41280-0.214,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222,I50_idx
0,1000015,1,0,1,0.11,NaN,0.12,1999-07-22,2018-06-07,2002-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.11
1,1000027,0,0,0,NaN,NaN,NaN,2015-01-16,2014-10-24,2014-10-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,NaN,NaN,NaN,2020-02-13,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN,1999-06-10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,NaN,NaN,NaN,2014-09-30,2014-09-30,2014-09-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,NaN,NaN,NaN,2013-12-30,2013-12-30,2013-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,NaN,NaN,NaN,1999-01-25,2015-10-17,2002-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
c = df5.columns.tolist()
for col in c: 
    print(col)

eid
H401
H402
H403
H404
H405
H406
H407
H408
H409
H401_index
H402_index
H403_index
H404_index
H405_index
H406_index
H407_index
H408_index
H409_index
41280-0.0
41280-0.1
41280-0.2
41280-0.3
41280-0.4
41280-0.5
41280-0.6
41280-0.7
41280-0.8
41280-0.9
41280-0.10
41280-0.11
41280-0.12
41280-0.13
41280-0.14
41280-0.15
41280-0.16
41280-0.17
41280-0.18
41280-0.19
41280-0.20
41280-0.21
41280-0.22
41280-0.23
41280-0.24
41280-0.25
41280-0.26
41280-0.27
41280-0.28
41280-0.29
41280-0.30
41280-0.31
41280-0.32
41280-0.33
41280-0.34
41280-0.35
41280-0.36
41280-0.37
41280-0.38
41280-0.39
41280-0.40
41280-0.41
41280-0.42
41280-0.43
41280-0.44
41280-0.45
41280-0.46
41280-0.47
41280-0.48
41280-0.49
41280-0.50
41280-0.51
41280-0.52
41280-0.53
41280-0.54
41280-0.55
41280-0.56
41280-0.57
41280-0.58
41280-0.59
41280-0.60
41280-0.61
41280-0.62
41280-0.63
41280-0.64
41280-0.65
41280-0.66
41280-0.67
41280-0.68
41280-0.69
41280-0.70
41280-0.71
41280-0.72
41280-0.73
41280-0.74
41280-0.75
41280-0.76
41280-0.77
4128

In [25]:
# 열별로 진단받은 날짜 추가하기

valuelist = df5['I50_idx'].tolist()
df5['I50_Date'] = list(map(lambda x, y: np.nan if y is np.nan else df5.loc[df5['eid'] == x, '41280-' + str(y)].values[0], df5['eid'], valuelist))

In [26]:
len(df5['I50_Date'].unique())

5075

In [27]:
df5

,eid,I500,I501,I509,I500_index,I501_index,I509_index,41280-0.0,41280-0.1,41280-0.2,...,41280-0.215,41280-0.216,41280-0.217,41280-0.218,41280-0.219,41280-0.220,41280-0.221,41280-0.222,I50_idx,I50_Date
0,1000015,1,0,1,0.11,NaN,0.12,1999-07-22,2018-06-07,2002-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.11,2018-06-07
1,1000027,0,0,0,NaN,NaN,NaN,2015-01-16,2014-10-24,2014-10-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,NaN,NaN,NaN,2020-02-13,2018-10-16,2018-10-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN,1999-06-10,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,NaN,NaN,NaN,2014-09-30,2014-09-30,2014-09-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,NaN,NaN,NaN,2013-12-30,2013-12-30,2013-12-30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,NaN,NaN,NaN,1999-01-25,2015-10-17,2002-03-24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df6 = df5[['eid', 
'I500', 'I501', 'I509', 
'I500_index', 'I501_index', 'I509_index', 
'I50_idx', 'I50_Date']]

In [29]:
df6

,eid,I500,I501,I509,I500_index,I501_index,I509_index,I50_idx,I50_Date
0,1000015,1,0,1,0.11,NaN,0.12,0.11,2018-06-07
1,1000027,0,0,0,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,NaN,NaN,NaN,NaN,NaN


In [71]:
# df6.to_csv('0. data\\ukb46987_H40preprocessed.csv', encoding = 'euc-kr', index = False)

#### 2021.12.06 ~

In [72]:
# df = pd.read_csv('0. data\\ukb46987_H40preprocessed.csv', encoding = 'euc-kr')
# df

,eid,H401,H402,H403,H404,H405,H406,H407,H408,H409,...,H402_index,H403_index,H404_index,H405_index,H406_index,H407_index,H408_index,H409_index,H40_idx,H40_Date
0,1000015,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000027,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000039,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000040,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000053,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502472,6025340,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502473,6025353,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502474,6025364,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
df = df6.copy()

In [31]:
# def conditions(df):
#     if (df['H400'] == 1) or (df['H401'] == 1) or (df['H402'] == 1) or (df['H403'] == 1) or (df['H404'] == 1) or (df['H405'] == 1) or (df['H406'] == 1) or (df['H407'] == 1) or (df['H408'] == 1) or (df['H409'] == 1):
#         return 1
#     else:
#         return 0

def conditions(df):
    if (df['I500'] == 1) or (df['I501'] == 1) or (df['I509'] == 1):
        return 1
    else:
        return 0

In [32]:
df['I50'] = df.apply(conditions, axis=1)

In [33]:
df['I50'].value_counts()

0    488536
1     13940
Name: I50, dtype: int64

In [34]:
df['I50_Date'].fillna(0, inplace = True)

In [35]:
df.loc[df['I50_Date'] == 0, 'I50_Date'].count()

488536

In [36]:
df

,eid,I500,I501,I509,I500_index,I501_index,I509_index,I50_idx,I50_Date,I50
0,1000015,1,0,1,0.11,NaN,0.12,0.11,2018-06-07,1
1,1000027,0,0,0,NaN,NaN,NaN,NaN,0,0
2,1000039,0,0,0,NaN,NaN,NaN,NaN,0,0
3,1000040,0,0,0,NaN,NaN,NaN,NaN,0,0
4,1000053,0,0,0,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...
502471,6025339,0,0,0,NaN,NaN,NaN,NaN,0,0
502472,6025340,0,0,0,NaN,NaN,NaN,NaN,0,0
502473,6025353,0,0,0,NaN,NaN,NaN,NaN,0,0
502474,6025364,0,0,0,NaN,NaN,NaN,NaN,0,0


In [37]:
df_final = df[['eid', 'I50_Date', 'I50']]
df_final

,eid,I50_Date,I50
0,1000015,2018-06-07,1
1,1000027,0,0
2,1000039,0,0
3,1000040,0,0
4,1000053,0,0
...,...,...,...
502471,6025339,0,0
502472,6025340,0,0
502473,6025353,0,0
502474,6025364,0,0


In [38]:
df_final['I50'].value_counts()

0    488536
1     13940
Name: I50, dtype: int64

In [39]:
df_final.to_csv('0. data\\ukb46987_I50preprocessed.csv', encoding = 'euc-kr', index = False)